In [27]:
# copy x train five times for each model
#different workflow patterns, (pca vs no pca, hyperopt vs no, high dim vs low)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
#from plotter import Plotter

### Data preperation

#### Load the data and generate descriptors

In [28]:
pest_data = pd.read_csv("Data/BinaryPesticideData.csv", delimiter=",")
pest_data = pest_data.drop(['LOD'], axis=1)
pest_data[0:5]

,Pesticide Name,EPA Tolerance (ppm),Smiles,ExactMolWt,HeavyAtomMolWt,NumRadicalElectrons,NumValenceElectrons,HeavyAtomCount,NHOHCount,NOCount,...,FeatureAcceptorCount3D,FeatureDonorCount3D,FeatureAnionCount3D,FeatureCationCount3D,FeatureRingCount3D,FeatureHydrophobeCount3D,ConformerModelRMSD3D,EffectiveRotorCount3D,ConformerCount3D,Class
0,Procymidone,NT,CC12CC1(C)C(=O)N(C2=O)c3cc(Cl)cc(Cl)c3,283.016684,273.054,0,94,18,0,3,...,2.0,0.0,0.0,0.0,2.0,0.0,0.6,1.4,3,1
1,Fenamiphos,NT,CCO[P](=O)(NC(C)C)Oc1ccc(SC)c(C)c1,303.105801,281.188,0,108,19,1,4,...,1.0,0.0,0.0,0.0,1.0,2.0,0.8,7.0,10,1
2,Metribuzin,NT,CSC1=NN=C(C(=O)N1N)C(C)(C)C,214.088832,200.182,0,78,14,2,5,...,2.0,1.0,0.0,0.0,1.0,2.0,0.6,2.0,3,1
3,Fenamiphos sulfone,NT,CCO[P](=O)(NC(C)C)Oc1ccc(c(C)c1)[S](C)(=O)=O,335.095630,313.186,0,120,21,1,6,...,3.0,0.0,0.0,0.0,1.0,1.0,0.8,7.0,10,1
4,Methiocarb,NT,CNC(=O)Oc1cc(C)c(SC)c(C)c1,225.082350,210.193,0,82,15,1,3,...,1.0,1.0,0.0,0.0,1.0,1.0,0.6,4.0,10,1


In [29]:
pest_data.tail()

,Pesticide Name,EPA Tolerance (ppm),Smiles,ExactMolWt,HeavyAtomMolWt,NumRadicalElectrons,NumValenceElectrons,HeavyAtomCount,NHOHCount,NOCount,...,FeatureAcceptorCount3D,FeatureDonorCount3D,FeatureAnionCount3D,FeatureCationCount3D,FeatureRingCount3D,FeatureHydrophobeCount3D,ConformerModelRMSD3D,EffectiveRotorCount3D,ConformerCount3D,Class
235,Oxamyl oxime,T,CSC(=N/O)/C(=O)N(C)C,162.046299,152.134,0,58,10,1,4,...,2.0,0.0,1.0,0.0,0.0,1.0,0.6,3.0,10,2
236,Pyriproxyfen,T,CC(COc1ccc(Oc2ccccc2)cc1)Oc3ccccn3,321.136493,302.224,0,122,24,0,4,...,4.0,0.0,0.0,0.0,3.0,0.0,1.0,7.0,10,2
237,Boscalid,T,Clc1ccc(cc1)c2ccccc2NC(=O)c3cccnc3Cl,342.032668,331.117,0,114,23,1,3,...,2.0,1.0,0.0,0.0,3.0,0.0,0.6,4.0,10,2
238,Fenpropathrin,T,CC1(C)C(C(=O)OC(C#N)c2cccc(Oc3ccccc3)c2)C1(C)C,349.167794,326.246,0,134,26,0,4,...,3.0,0.0,0.0,0.0,2.0,2.0,1.0,6.6,10,2
239,Carfentrazone ethyl,T,CCOC(=O)C(Cl)Cc1cc(N2N=C(C)N(C(F)F)C2=O)c(F)cc1Cl,411.036431,398.083,0,142,26,0,6,...,2.0,0.0,0.0,0.0,2.0,0.0,0.8,7.0,10,2


In [4]:
pest_data.count()

Pesticide Name              240
EPA Tolerance (ppm)         240
Smiles                      240
ExactMolWt                  240
HeavyAtomMolWt              240
NumRadicalElectrons         240
NumValenceElectrons         240
HeavyAtomCount              240
NHOHCount                   240
NOCount                     240
NumAliphaticCarbocycles     240
NumAliphaticHeterocycles    240
NumAliphaticRings           240
NumAromaticCarbocycles      240
NumAromaticHeterocycles     240
NumAromaticRings            240
NumHAcceptors               240
NumHDonors                  240
NumHeteroatoms              240
NumRotatableBonds           240
NumSaturatedCarbocycles     240
NumSaturatedHeterocycles    240
NumSaturatedRings           240
RingCount                   240
Halogens                    240
MolecularFormula            240
MolecularWeight             240
CanonicalSMILES             240
XLogP                       240
ExactMass                   240
TPSA                        240
Complexi

In [5]:
pest_data["Class"].dtype

dtype('int64')

In [6]:
pest_data.columns

Index(['Pesticide Name', 'EPA Tolerance (ppm)', 'Smiles', 'ExactMolWt',
       'HeavyAtomMolWt', 'NumRadicalElectrons', 'NumValenceElectrons',
       'HeavyAtomCount', 'NHOHCount', 'NOCount', 'NumAliphaticCarbocycles',
       'NumAliphaticHeterocycles', 'NumAliphaticRings',
       'NumAromaticCarbocycles', 'NumAromaticHeterocycles', 'NumAromaticRings',
       'NumHAcceptors', 'NumHDonors', 'NumHeteroatoms', 'NumRotatableBonds',
       'NumSaturatedCarbocycles', 'NumSaturatedHeterocycles',
       'NumSaturatedRings', 'RingCount', 'Halogens', 'MolecularFormula',
       'MolecularWeight', 'CanonicalSMILES', 'XLogP', 'ExactMass', 'TPSA',
       'Complexity', 'Charge', 'HBondDonorCount', 'HBondAcceptorCount',
       'RotatableBondCount', 'HeavyAtomCount.1', 'IsotopeAtomCount',
       'AtomStereoCount', 'DefinedAtomStereoCount', 'UndefinedAtomStereoCount',
       'BondStereoCount', 'DefinedBondStereoCount', 'UndefinedBondStereoCount',
       'CovalentUnitCount', 'Volume3D', 'XStericQuadrupol

In [7]:
pest_data.describe()

,ExactMolWt,HeavyAtomMolWt,NumRadicalElectrons,NumValenceElectrons,HeavyAtomCount,NHOHCount,NOCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,...,FeatureAcceptorCount3D,FeatureDonorCount3D,FeatureAnionCount3D,FeatureCationCount3D,FeatureRingCount3D,FeatureHydrophobeCount3D,ConformerModelRMSD3D,EffectiveRotorCount3D,ConformerCount3D,Class
count,240.000000,240.000000,240.0,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,...,240.000000,240.00000,240.000000,240.000000,240.000000,240.000000,240.000000,240.00000,240.000000,240.000000
mean,296.822371,282.050196,0.0,103.650000,19.129167,0.654167,4.279167,0.166667,0.183333,0.350000,...,1.866667,0.53750,0.100000,0.441667,1.562500,0.712500,0.741667,5.40000,8.604167,1.708333
std,81.867931,79.940445,0.0,29.072309,5.705828,0.892310,1.641889,0.445025,0.398393,0.680266,...,1.267114,0.73656,0.374948,0.735198,0.974706,0.908266,0.206011,2.57331,2.774206,0.455480
min,44.026215,40.021000,0.0,18.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.400000,0.00000,1.000000,1.000000
25%,237.882259,222.210000,0.0,82.000000,15.000000,0.000000,3.000000,0.000000,0.000000,0.000000,...,1.000000,0.00000,0.000000,0.000000,1.000000,0.000000,0.600000,3.95000,10.000000,1.000000
50%,299.589972,282.830500,0.0,104.000000,19.000000,0.000000,4.000000,0.000000,0.000000,0.000000,...,2.000000,0.00000,0.000000,0.000000,2.000000,0.000000,0.800000,5.90000,10.000000,2.000000
75%,346.802600,327.363750,0.0,122.500000,23.000000,1.000000,5.000000,0.000000,0.000000,1.000000,...,3.000000,1.00000,0.000000,1.000000,2.000000,1.000000,0.800000,7.00000,10.000000,2.000000
max,682.023324,660.220000,0.0,210.000000,38.000000,5.000000,10.000000,2.000000,2.000000,3.000000,...,6.000000,4.00000,2.000000,3.000000,4.000000,6.000000,1.400000,13.00000,10.000000,2.000000


In [8]:
#pest_data.hist(column='Na_to_K', bins=100)

In [9]:
#pest_data.hist(column='Age', bins=50)

In [30]:
X = pest_data[[#'LOD',
               'ExactMolWt', 'HeavyAtomMolWt', 'NumValenceElectrons', 'HeavyAtomCount', 'NHOHCount', 
                        'NOCount', 'NumAliphaticCarbocycles', 'NumAliphaticHeterocycles', 'NumAliphaticRings', 
                        'NumAromaticCarbocycles', 'NumAromaticHeterocycles', 'NumAromaticRings', 'NumHAcceptors', 
                        'NumHDonors', 'NumHeteroatoms', 'NumRotatableBonds', 'NumSaturatedCarbocycles', 
                        'NumSaturatedHeterocycles', 'NumSaturatedRings', 'RingCount', 'Halogens', "MolecularWeight", 
                        "XLogP", "ExactMass", "TPSA", "Complexity", "Charge", "HBondDonorCount", 
                        "HBondAcceptorCount", "RotatableBondCount", "HeavyAtomCount", "IsotopeAtomCount", "AtomStereoCount", 
                        "DefinedAtomStereoCount", "UndefinedAtomStereoCount", "BondStereoCount", "DefinedBondStereoCount", 
                        "UndefinedBondStereoCount", "CovalentUnitCount","Volume3D", "XStericQuadrupole3D", "YStericQuadrupole3D", 
                        "ZStericQuadrupole3D", "FeatureCount3D", "FeatureAcceptorCount3D", "FeatureDonorCount3D", "FeatureAnionCount3D", 
                        "FeatureCationCount3D", "FeatureRingCount3D", "FeatureHydrophobeCount3D", "ConformerModelRMSD3D", 
                        "EffectiveRotorCount3D", "ConformerCount3D"]].values


y = pest_data[['Class']].values
#y[0:5]

X[0:2]

array([[283.01668395, 273.054     ,  94.        ,  18.        ,
          0.        ,   3.        ,   1.        ,   1.        ,
          2.        ,   1.        ,   0.        ,   1.        ,
          2.        ,   0.        ,   5.        ,   1.        ,
          1.        ,   1.        ,   2.        ,   3.        ,
          2.        , 284.13      ,   3.        , 283.016684  ,
         37.4       , 405.        ,   0.        ,   0.        ,
          2.        ,   1.        ,  18.        ,   0.        ,
          2.        ,   0.        ,   2.        ,   0.        ,
          0.        ,   0.        ,   1.        , 203.3       ,
          7.89      ,   2.66      ,   0.89      ,   4.        ,
          2.        ,   0.        ,   0.        ,   0.        ,
          2.        ,   0.        ,   0.6       ,   1.4       ,
          3.        ],
       [303.10580119, 281.188     , 108.        ,  19.        ,
          1.        ,   4.        ,   0.        ,   0.        ,
          0.     

In [31]:
y_array = pest_data[['Class']].to_numpy()
y_array[0:5]

array([[1],
       [1],
       [1],
       [1],
       [1]], dtype=int64)

In [12]:
pest_feats_array = X
y = y_array

In [39]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

pest_feats_scaled = StandardScaler().fit_transform(pest_feats_array)  # Previously, a huge bias without scaling

# one feature had 99% explanation

pest_feats_pca = PCA(n_components=18).fit(pest_feats_scaled)
pest_feats_pca_values = pest_feats_pca.transform(pest_feats_scaled)
pest_feats_pca_values

array([[-0.37672096,  7.35146325, -1.36969771, ...,  0.86462008,
         0.63763199, -0.44582798],
       [ 0.23188303, -0.83649599,  1.86547439, ...,  0.50090363,
        -0.81411582, -0.22069463],
       [-4.02410103, -1.42806754, -1.06507277, ...,  0.39073028,
        -0.90132833, -1.54905889],
       ...,
       [ 1.25873943,  0.42172181, -3.03086217, ..., -0.56262534,
         0.91421341, -0.68821212],
       [ 4.52998284,  2.87163471,  0.4030525 , ..., -1.1250404 ,
        -0.07063966, -0.45797793],
       [ 4.74075739, -1.47960568, -0.0307702 , ..., -0.30309032,
         1.04193355,  0.18330613]])

In [40]:
pest_feats_pca.explained_variance_ratio_

array([0.30427204, 0.12770474, 0.10133478, 0.08528545, 0.0581771 ,
       0.04446786, 0.03999223, 0.0322997 , 0.02871666, 0.02768237,
       0.02099046, 0.01982549, 0.01658918, 0.01590529, 0.01215478,
       0.01065372, 0.00839736, 0.00781981])

In [41]:
pest_feats_pca.explained_variance_ratio_.sum()

0.962269022505085

In [42]:
#pest_data['Class'] = pest_data['Class'].astype(int)
#y = np.asanyarray(pest_data['Class'])
#y[0:5]

X = pest_feats_pca_values[:]
X.shape

(240, 18)

#### train_test_split

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=True) #random_state = 3
print ('X_train and y_train shape: ', X_train.shape, y_train.shape)
print ('X_test and y_test shape: ', X_test.shape, y_test.shape)

X_train and y_train shape:  (192, 53) (192, 1)
X_test and y_test shape:  (48, 53) (48, 1)


In [33]:
from sklearn.preprocessing import StandardScaler

#X_train = StandardScaler().fit_transform(X_train)
X_train[0:5]
#X_test = StandardScaler().fit_transform(X_test)
X_test[0:2]

array([[3.28203845e+02, 3.00228000e+02, 1.30000000e+02, 2.40000000e+01,
        0.00000000e+00, 3.00000000e+00, 2.00000000e+00, 0.00000000e+00,
        2.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        3.00000000e+00, 0.00000000e+00, 3.00000000e+00, 6.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 1.00000000e+00, 2.00000000e+00,
        0.00000000e+00, 3.28400000e+02, 5.40000000e+00, 3.28203845e+02,
        4.34000000e+01, 6.42000000e+02, 0.00000000e+00, 0.00000000e+00,
        3.00000000e+00, 7.00000000e+00, 2.40000000e+01, 0.00000000e+00,
        3.00000000e+00, 3.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 1.00000000e+00, 2.78600000e+02,
        2.09300000e+01, 1.64000000e+00, 1.24000000e+00, 6.00000000e+00,
        2.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 3.00000000e+00, 8.00000000e-01, 8.00000000e+00,
        1.00000000e+01],
       [2.55922578e+02, 2.49373000e+02,

In [34]:
#Just to be sure

if X_train.shape[0] == y_train.shape[0]:
    print("Train set dimensions match and is equal to", X_train.shape, 'vs', y_train.shape)
else:
    print("Train set dimensions do not match")

Train set dimensions match and is equal to (192, 53) vs (192, 1)


### Decision Tree HyperOpt - Optimisation of model hyperparameters

In [35]:
from hpsklearn import HyperoptEstimator, any_classifier, any_preprocessing #any_sparse_classifier
from hyperopt import fmin, tpe
from hpsklearn import decision_tree_classifier

# preprocessing = []

model_DT = HyperoptEstimator(classifier=decision_tree_classifier('myDT'), algo=tpe.suggest, max_evals=300, trial_timeout=500)

# perform the search
model_DT.fit(X_train, y_train.ravel())

 96%|██████████████████████████████████████████████████████████████████████▊   | 22/23 [00:08<?, ?trial/s, best loss=?]


In [21]:
accuracy_DT = model_DT.score(X_test, y_test)
print("Accuracy_DT: %.3f" % accuracy_DT)

# model details
print(model_DT.best_model())

Accuracy_DT: 0.667
{'learner': DecisionTreeClassifier(criterion='entropy', max_features=0.5143712667457783,
                       random_state=4, splitter='random'), 'preprocs': (StandardScaler(),), 'ex_preprocs': ()}


In [22]:
from sklearn.metrics import classification_report, accuracy_score
y_hat_DT = model_DT.predict(X_test)
print(classification_report(y_test, y_hat_DT))

              precision    recall  f1-score   support

           1       0.23      0.33      0.27         9
           2       0.83      0.74      0.78        39

    accuracy                           0.67        48
   macro avg       0.53      0.54      0.53        48
weighted avg       0.72      0.67      0.69        48



#### Cross-validation for better accuracy

In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

model = DecisionTreeClassifier(criterion="gini", splitter='best', max_depth = 12)

score = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy", n_jobs=-1).mean()
print(score)

0.5569500674763833


In [24]:
model = DecisionTreeClassifier(criterion="entropy", splitter='best', max_depth = 12)

score = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy", n_jobs=-1).mean()
print(score)

0.578272604588394


In [26]:
import pickle
with open('pesticide_dt_model.pkl', 'wb') as file:
    pickle.dump(model_DT, file)